In [ ]:
pip install python_speech_features

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5886 sha256=215f431061833e242dc4741f92039bad81486d87bba0e971dd31dfb014a6b372
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
Successfully built python_speech_features


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

# Set the path to the librispeech folder
librispeech_path = "/content/drive/MyDrive/LibriSpeech"

Mounted at /content/drive


In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'


In [ ]:
#GPU preparation
from keras.backend import set_session
import tensorflow as tf 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
  except RuntimeError as e:
    print(e)
%load_ext autoreload
%autoreload 2
from sample_models import *
from train_utils import train_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
model_2 = cnn_rnn_model(input_dim=161, 
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='valid',
                        units=200)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 161)]       0         
                                                                 
 conv1d (Conv1D)             (None, None, 200)         354400    
                                                                 
 bn_conv_1d (BatchNormalizat  (None, None, 200)        800       
 ion)                                                            
                                                                 
 rnn (CuDNNGRU)              (None, None, 200)         241200    
                                                                 
 batch_normalization (BatchN  (None, None, 200)        800       
 ormalization)                                                   
                                                                 
 time_distributed (TimeDistr  (None, None, 29)         5829  

In [ ]:
train_model(input_to_softmax=model_2, 
            pickle_path='model_2.pickle', 
            save_model_path='model_2.h5', 
            spectrogram=True) 

/content/train_utils.py:74: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(generator=audio_gen.next_train(), steps_per_epoch=steps_per_epoch,


Epoch 1/30
106/106 [==============================] - 3285s 31s/step - loss: 263.4915 - val_loss: 224.4163
Epoch 2/30
106/106 [==============================] - 101s 956ms/step - loss: 178.3881 - val_loss: 171.2186
Epoch 3/30
106/106 [==============================] - 84s 792ms/step - loss: 158.8696 - val_loss: 154.4125
Epoch 4/30
106/106 [==============================] - 82s 776ms/step - loss: 149.5786 - val_loss: 143.8856
Epoch 5/30
106/106 [==============================] - 80s 754ms/step - loss: 142.8584 - val_loss: 144.1737
Epoch 6/30
106/106 [==============================] - 79s 746ms/step - loss: 140.0500 - val_loss: 139.4100
Epoch 7/30
106/106 [==============================] - 84s 798ms/step - loss: 134.8718 - val_loss: 141.4298
Epoch 8/30
106/106 [==============================] - 85s 800ms/step - loss: 132.8446 - val_loss: 142.8406
Epoch 9/30
106/106 [==============================] - 79s 752ms/step - loss: 130.2332 - val_loss: 139.4320
Epoch 10/30
106/106 [===============

In [ ]:
import numpy as np
from data_generator import AudioGenerator
from keras import backend as K
from utils import int_sequence_to_text
from IPython.display import Audio

def get_predictions(index, partition, input_to_softmax, model_path):
    # load the train and test data
    data_gen = AudioGenerator()
    data_gen.load_train_data()
    data_gen.load_validation_data()
    
    # obtain  transcription and the audio features 
    if partition == 'validation':
        transcr = data_gen.valid_texts[index]
        audio_path = data_gen.valid_audio_paths[index]
        data_point = data_gen.normalize(data_gen.featurize(audio_path))
    elif partition == 'train':
        transcr = data_gen.train_texts[index]
        audio_path = data_gen.train_audio_paths[index]
        print(audio_path)
        data_point = data_gen.normalize(data_gen.featurize(audio_path))
    else:
        raise Exception('Invalid partition!  Must be "train" or "validation"')
        
    # obtain and decode the  model's predictions
    input_to_softmax.load_weights(model_path)
    prediction = input_to_softmax.predict(np.expand_dims(data_point, axis=0))
    output_length = [input_to_softmax.output_length(data_point.shape[0])] 
    pred_ints = (K.eval(K.ctc_decode(
                prediction, output_length)[0][0])+1).flatten().tolist()
    print(pred_ints)
    
    # play the audio file, and display the true and predicted transcriptions
    print('-'*80)
    Audio(audio_path)
    print('True transcription:\n' + '\n' + transcr)
    print('-'*80)
    print('Predicted transcription:\n' + '\n' + ''.join(int_sequence_to_text(pred_ints)))
    print('-'*80)

In [ ]:
get_predictions(index=200, 
                partition='train',
                input_to_softmax=model_2, 
                model_path='results/model_2.h5')

/content/drive/MyDrive/LibriSpeech/dev-clean/6241/66616/6241-66616-0013.wav
1/1 [==============================] - 0s 110ms/step
[17, 21, 2, 18, 7, 4, 17, 2, 22, 10, 7, 2, 21, 17, 15, 16, 2, 14, 17, 2, 14, 17, 17, 2, 18, 10, 7, 2, 25, 17, 11, 21, 2, 4, 7, 3, 21, 18, 3, 5, 11, 17, 14, 7, 2, 6, 3, 7, 2, 17, 16, 3, 2, 6, 7, 18, 20, 7, 6, 3, 5, 11, 17, 16, 2, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# specify the model
model_end = final_model(input_dim=161, 
                        units=200,
                        filters=200,
                        kernel_size=11, 
                        conv_stride=2,
                        conv_border_mode='valid',
                        recur_layers=2)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 161)]       0         
                                                                 
 conv1d (Conv1D)             (None, None, 200)         354400    
                                                                 
 bn_conv_1d (BatchNormalizat  (None, None, 200)        800       
 ion)                                                            
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 200)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, None, 200)         0         
                                                                 
 bidirectional (Bidirectiona  (None, None, 400)        4824

In [ ]:
get_predictions(index=1, 
                partition='train',
                input_to_softmax=model_end, 
                model_path='results/model_end.h5')

/content/drive/MyDrive/LibriSpeech/dev-clean/6295/64301/6295-64301-0002.wav
1/1 [==============================] - 4s 4s/step
[25, 3, 21, 2, 22, 10, 20, 2, 7, 24, 7, 20, 2, 10, 2, 10, 3, 18, 7, 20, 2, 15, 3, 16, 2, 22, 10, 3, 16, 2, 12, 17, 21, 17, 10, 2, 22, 10, 3, 22, 2, 16, 11, 2, 11, 21, 2, 7, 2, 21, 22, 20, 17, 6, 2, 3, 14, 17, 16, 2, 10, 2, 8, 17, 2, 18, 3, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------
True transcription:

was there ever a happier man than joseph that night as he strode along the footpath
--------------------------------------------------------------------------------
Predicted transcription:

was thr ever h haper man than josoh that ni is

In [ ]:
get_predictions(index=0, 
                partition='validation',
                input_to_softmax=model_end, 
                model_path='results/model_end.h5')

1/1 [==============================] - 0s 92ms/step
[22, 10, 7, 16, 27, 10, 7, 20, 2, 20, 3, 21, 10, 6, 17, 16, 21, 2, 21, 22, 7, 7, 20, 21, 2, 16, 2, 22, 17, 2, 14, 7, 5, 17, 20, 11, 22, 27, 2, 17, 20, 6, 2, 21, 10, 17, 25, 6, 11, 16, 9, 2, 14, 2, 22, 14, 27, 2, 8, 17, 20, 2, 7, 2, 21, 17, 14, 6, 7, 20, 21, 17, 16, 2, 22, 20, 11, 16, 11, 16, 9, 2, 22, 17, 2, 18, 3, 22, 5, 22, 2, 7, 24, 7, 20, 27, 2, 17, 16, 2, 11, 16, 9, 2, 3, 21, 2, 22, 11, 15, 2, 15, 11, 16, 2, 7, 21, 2, 17, 22, 2, 22, 10, 2, 21, 17, 14, 7, 2, 15, 7, 2, 25, 3, 21, 2, 16, 17, 20, 27, 2, 5, 3, 23, 22, 10, 11, 17, 16, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------------------------------------------------------------------
True transcription:

then he rushed down stairs into the courty